In [ ]:
from PIL import Image, ImageDraw, ImageFont, ImageOps
import os
import requests
from io import BytesIO
import json

In [111]:
# --- Configuration ---
TEMPLATE_PATH = 'template.png'
OUTPUT_DIR = 'cards'
FONT_PATH = 'fonts/Arimo-VariableFont_wght.ttf' # Path to a .ttf font file (e.g., download from Google Fonts or use one installed on your OS)

# Define positions and sizes on your template (these are examples, adjust for your template)
# All coordinates are (x, y) from the top-left corner of the image.
# Image positions are top-left corners.
# Text positions are usually top-left of the text bounding box.
CARD_DIMS = (1080, 1920)
PIC_MAX = (580, 670)
POSITIONS = {
    'pic_pos': (75, 250),      # Top-left corner to paste player's face
    'name_pos': (0, 80),           # Position for player's name
    'partyName_pos': (700, 300),      # Position for Strength stat
    'state_pos': (700, 400),         # Position for Speed stat
    'chamber_pos': (700, 500),       # Position for Agility stat
    'endYear_pos': (700, 600),       # Position for Overall stat

}

# Font sizes (adjust as needed)
FONT_SIZES = {
    'name': 70,
    'stats':30,
    'labels': 40
}



In [ ]:
# --- Player Data (as defined above) ---
with open('congressmen.json', 'r') as f:
    player_data = json.load(f)
"""
player_data = [
    {
        'name': 'Alexandria Ocasio-Cortez',
        'pic': 'https://www.congress.gov/img/member/o000172_200.jpg',
        'partyName': 'Democrat',
        'state': 'New York',
        'chamber': 'House of Representatives',
        'endYear': 2026
    }
]"""

# --- Load fonts ---
try:
    font_name = ImageFont.truetype(FONT_PATH, FONT_SIZES['name'])
    font_stats = ImageFont.truetype(FONT_PATH, FONT_SIZES['stats'])
    font_labels = ImageFont.truetype(FONT_PATH, FONT_SIZES['labels'])
except IOError:
    print(f"Warning: Could not load font from {FONT_PATH}. Using default Pillow font. "
        "Ensure the font file exists and is accessible.")
    font_name = ImageFont.load_default()
    font_stats = ImageFont.load_default()
    font_labels = ImageFont.load_default()

In [171]:
print(player_data)

[{'bioguideID': 'C000243', 'name': 'Castle, Michael N.', 'partyName': 'Republican', 'state': 'Delaware', 'url': 'https://api.congress.gov/v3/member/C000243?format=json', 'attribution': 'Collection of the U.S. House of Representatives', 'imageUrl': 'https://www.congress.gov/img/member/c000243_200.jpg', 'chamber': 'House of Representatives', 'endYear': 2011, 'startYear': 1993}, {'bioguideID': 'G000590', 'name': 'Green, Mark E.', 'partyName': 'Republican', 'state': 'Tennessee', 'url': 'https://api.congress.gov/v3/member/G000590?format=json', 'attribution': 'Image courtesy of the Member', 'imageUrl': 'https://www.congress.gov/img/member/g000590_200.jpg', 'chamber': 'House of Representatives', 'endYear': 2025, 'startYear': 2019}, {'bioguideID': 'C000488', 'name': 'Clay, William (Bill)', 'partyName': 'Democratic', 'state': 'Missouri', 'url': 'https://api.congress.gov/v3/member/C000488?format=json', 'attribution': 'Collection of the U.S. House of Representatives', 'imageUrl': 'https://www.con

In [96]:
def center_text(draw, text, font, text_color, offset=(0,0), center=(True,True)):

    _, _, w, h = draw.textbbox((0,0), text=text, font=font)
    #image_width, image_height = card.size
    if center[0]:
        x_text = ((CARD_DIMS[0] - offset[0] - w) / 2 ) + offset[0]
    else: 
        x_text = offset[0]
    if center[1]:
        y_text = ((CARD_DIMS[1] - offset[1] - h) / 2 ) + offset[1]

    else:
        y_text = offset[1]
    draw.text((x_text, y_text), text=text, font=font, fill=text_color)



In [ ]:
def pull_pic_from_web(rep):
    """
    Pull the photo from the web link. 
    If no link was given, fill with an empty photo.
    
    Args:
        rep (dict): Dictionary with rep info 
    Returns:
        (img): Image of rep's face
    """

    try:
        face_path = rep['imageUrl']
        if "http" in face_path:
            #Pull from web source
            print(f"Found face image at URL {face_path}. Saving.")
            face_path_url = requests.get(face_path)
            img = Image.open(BytesIO(face_path_url.content))
        else:
            print (f"Not sure what format this photo is in: {face_path}. Creating dummy face image.")
            img = Image.new('RGB', PIC_MAX, color = 'lightgray')

    except ImportError:
        print(f"Pillow is installed, but couldn't create dummy images. Please ensure {FONT_PATH} or a similar font is available, or manually create face images.")
        pass # Continue without dummy images if PIL or font issues persist
    except Exception as e:
        print("Likely couldn't find 'imageUrl', fix the dict")

    return img




In [118]:
def create_text_box(draw, message, font, text_color=(0,0,0), box_corners=(0,0,100,100), output_path="textbox_example.png"):
    """

    Args:
        draw [draw]: input drawing
        message [str]: Input message 
        font : Input font
        box_corners [length 4 tuple]: defaults to upper left corner, should be upper left and lower right
            corners of text box
        text_color [length 3 tuple]: color of text, defaults to black
        
    """

    # Define text box properties
    box_x1, box_y1 = (box_corners[0], box_corners[1])
    box_x2, box_y2 = (box_corners[2], box_corners[3])


    # Draw the rectangle for the text box background
    draw.rectangle([box_x1, box_y1, box_x2, box_y2])

    # Calculate text position to center it within the box (optional)
    # Note: getbbox() for multiline text returns bounding box for the entire block
    # For more precise centering, you might need to calculate line by line.
    text_bbox = draw.textbbox((0,0), message, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]

    text_x = box_x1 + (box_x2 - box_x1 - text_width) / 2
    text_y = box_y1 + (box_y2 - box_y1 - text_height) / 2

    # Draw the text on top of the rectangle
    draw.multiline_text((text_x, text_y), message, fill=text_color, font=font, align="center")



In [159]:
def draw_wrapped_text(draw_context, text, font, xy, max_width, fill_color=(0, 0, 0)):
    """
    Draws text with automatic wrapping and handles existing newline characters.

    Args:
        draw_context (ImageDraw.ImageDraw): The ImageDraw context to use for drawing.
        text (str): The string to draw, potentially containing "\n" characters.
        font (ImageFont.FreeTypeFont): The font to use.
        xy (tuple): A tuple of (x, y) coordinates for the top-left corner of the text box.
        max_width (int): The maximum width of the text box in pixels.
        fill_color (tuple): The color of the text (e.g., (0, 0, 0) for black).
    """
    x, y = xy
    all_lines = []
    
    # Split the initial text by newlines to handle pre-existing breaks
    paragraphs = text.split('\n')
    
    for para in paragraphs:
        lines = []
        line_words = []
        words = para.split(' ')

        for word in words:
            current_line = ' '.join(line_words + [word])
            text_bbox = draw_context.textbbox((0, 0), current_line, font=font)
            line_width = text_bbox[2] - text_bbox[0]

            if line_width > max_width and line_words:
                lines.append(' '.join(line_words))
                line_words = [word]
            else:
                line_words.append(word)

        if line_words:
            lines.append(' '.join(line_words))
            
        all_lines.extend(lines)

    # Now, draw each line one by one
    _, _, _, line_height = draw_context.textbbox((0,0), "A", font=font)
    
    for line in all_lines:
        draw_context.text((x, y), line, font=font, fill=fill_color)
        y += line_height * 1.2 # Move down to the next line with some padding



In [166]:

# --- Function to create a single player card ---
def create_card(rep_info, face_img):
    
    # 1. Open the template image
    card = Image.open(TEMPLATE_PATH).convert("RGBA") # Convert to RGBA for transparency handling

    # 2. Open and process the player's face image
    #face_img = ImageOps.fit(face_img, PIC_MAX, method=0, bleed=0.0, centering=(0.5, 0.5))
    face_img = face_img.resize(PIC_MAX) # Resize to desired dimensions

    print("Successfully resized face image")

    # Paste the face image onto the card
    card.paste(face_img, box=POSITIONS['pic_pos'])

    # 3. Prepare to draw text
    draw = ImageDraw.Draw(card)
    text_color = (0, 0, 0, 255) # Black color with full opacity

    # 4. Draw player name, centered
    center_text(draw, text=rep_info['name'], font=font_name, offset=POSITIONS['name_pos'], text_color=text_color, center=(True,False))

    # 5. Draw stats and labels
    # You might want to pre-draw labels on the template for static text like "Strength:"
    # Or draw them dynamically here.
    # For dynamic labels:

    message1 = rep_info['chamber'] + "\n\nParty: \n" + rep_info['partyName'] + \
        "\n\nState: \n" + rep_info['state'] + "\n\nUp for Re-election in " + str(rep_info['endYear'])

    draw_wrapped_text(draw, message1, font_labels, (690, 250), 300)


    # 6. Save the final card
    output_filename = os.path.join(OUTPUT_DIR, f"{rep_info['name'].replace(' ', '_').lower()}_card.png")
    card.save(output_filename)
    print(f"Created card: {output_filename}")

In [167]:
# Create a dummy template image if it doesn't exist
if not os.path.exists(TEMPLATE_PATH):
    print("\nExiting. Please set up your template and data, then run again.")

else:
    # --- Create output directory if it doesn't exist ---
    os.makedirs(OUTPUT_DIR, exist_ok=True) #Make the cards directory
    print(f"Using template: {TEMPLATE_PATH}")
    for rep in player_data:
        face_img = pull_pic_from_web(rep)
        create_card(rep, face_img)
    print("\nPlayer card generation complete!")

Using template: template.png
Found face image at URL https://www.congress.gov/img/member/o000172_200.jpg. Saving.
Successfully resized face image
Created card: cards\alexandria_ocasio-cortez_card.png

Player card generation complete!
